In [ ]:
# 2021A 3)结果

In [395]:
import numpy as np

alpha = 36.795 * np.pi / 180
beta = 78.168 * np.pi / 180
R = 300.4
F = 0.466 * R
focus_r = R - F
incoming = np.array([np.cos(beta) * np.cos(alpha), np.cos(beta) * np.sin(alpha), np.sin(beta)])

In [396]:
F

139.9864

In [397]:
incoming

array([0.16419486, 0.1228111 , 0.97875302])

In [398]:
"""beta_rotation_matrix, from S' to S"""
beta_rotation_matrix = np.array([
    [np.cos(beta), 0, -np.sin(beta)],
    [0, 1, 0],
    [np.sin(beta), 0, np.cos(beta)]
])
"""alpha_rotation_matrix, from S' to S"""
alpha_rotation_matrix = np.array([
    [np.cos(alpha), -np.sin(alpha), 0],
    [np.sin(alpha), np.cos(alpha), 0],
    [0, 0, 1]
])
"""reverse beta_rotation_matrix, from S to S'"""
beta_rotation_matrix_reverse = beta_rotation_matrix.T
"""reverse alpha_rotation_matrix, from S to S'"""
alpha_rotation_matrix_reverse = alpha_rotation_matrix.T

SRotateTo = beta_rotation_matrix_reverse @ alpha_rotation_matrix_reverse
SPRotateTo = alpha_rotation_matrix @ beta_rotation_matrix

In [399]:
SRotateTo @ incoming

array([1.00000000e+00, 1.38777878e-17, 0.00000000e+00])

In [400]:
SRotateTo

array([[ 0.16419486,  0.1228111 ,  0.97875302],
       [-0.59895372,  0.80078364,  0.        ],
       [-0.78376941, -0.58622776,  0.20504272]])

In [401]:
def move_to(p, d):
    x_p, y_p, z_p = p[0], p[1], p[2]
    # the new position should be a parabola with its vertex at vertex, and focus at focus
    # the node could only move along the radial direction
    point_in_S = np.array([x_p, y_p, z_p])
    point_in_SP = (SRotateTo @ point_in_S)
    Fd = F - d
    Rd = R - d
    x_n, y_n, z_n = point_in_SP[0], point_in_SP[1], point_in_SP[2]
    y2_z2 = y_n**2 + z_n**2
    ld = 2 * (Fd * x_n + np.sqrt(Fd * (Fd * x_n ** 2 + Rd * y2_z2))) / y2_z2
    new = ld * point_in_S
    original_length = np.sqrt(x_p**2 + y_p**2 + z_p**2)
    if original_length * ld > 0.6:
        return x_p, y_p, z_p
    return new[0], new[1], new[2]

In [402]:
move_to([0, 0, -300.4], 0.1)

(0, 0, -300.4)

In [403]:
incoming * 60.95 / incoming[0]

array([ 60.95      ,  45.58813048, 363.31830017])

In [404]:
SRotateTo @ incoming

array([1.00000000e+00, 1.38777878e-17, 0.00000000e+00])

In [405]:
incoming * 6095 / 16

array([ 62.54797883,  46.7833539 , 372.84373006])

In [406]:
move_to(incoming, 0)

(0.1641948582887741, 0.12281110130644991, 0.9787530239582539)

In [407]:
import pandas as pd

In [408]:
points = pd.read_csv("./附件1.csv")
points = points.rename(columns={
    # X坐标（米）,Y坐标（米）,Z坐标（米）
    "节点编号": "id",
    "X坐标（米）": "x",
    "Y坐标（米）": "y",
    "Z坐标（米）": "z",
})
points.head()

,id,x,y,z
0,A0,0.0000,0.000,-300.4000
1,B1,6.1078,8.407,-300.2202
2,C1,9.8827,-3.211,-300.2202
3,D1,0.0000,-10.391,-300.2202
4,E1,-9.8827,-3.211,-300.2202


In [409]:
def in_300(x, y, z):
    x_new, y_new, z_new = SRotateTo @ np.array([x, y, z])
    return y_new ** 2 + z_new ** 2 < 150 ** 2

mask = points.apply(lambda x: in_300(x['x'], x['y'], x['z']), axis=1)
points = points[mask]

In [410]:
import plotly.graph_objects as go

go.Figure(data=[
    go.Scatter3d(
        x = points['x'],
        y = points['y'],
        z = points['z'],
        mode = 'markers',
        marker=dict(
            size=2,
            color=points['z'],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
        )
    )
])

In [411]:
tris = pd.read_csv("./附件3.csv")
tris = tris.rename(columns={
    #主索节点1,主索节点2,主索节点3
    "主索节点1": "a",
    "主索节点2": "b",
    "主索节点3": "c",
})
tris.head()

,a,b,c
0,A0,B1,C1
1,A0,B1,A1
2,A0,C1,D1
3,A0,D1,E1
4,A0,E1,A1


In [412]:
valid_ids = points["id"].to_list()

In [413]:
def in_valid_ids(row):
    return (
        (row['a'] in valid_ids) and
        (row['b'] in valid_ids) and
        (row['c'] in valid_ids)
    )

In [414]:
tris = tris[tris.apply(in_valid_ids, axis=1)]

In [415]:
receiver = -incoming * focus_r
receiver

array([ -26.33908832,  -19.70057088, -157.00529608])

In [416]:
def intersection(normal_vector, plain_point, line_point, line_direction):
    """
    Computes the intersection point between a plane and a line in 3D space.
    
    Parameters:
    normal_vector (numpy array): A 3D vector representing the normal vector of the plane.
    plain_point (numpy array): A 3D vector representing a point on the plane.
    line_point (numpy array): A 3D vector representing a point on the line.
    line_direction (numpy array): A 3D vector representing the direction of the line.
    
    Returns:
    numpy array: A 3D vector representing the intersection point between the plane and the line.
    """
    # Compute the denominator of the t parameter
    denominator = np.dot(normal_vector, line_direction)
    
    # Check if the line is parallel to the plane
    if abs(denominator) < 1e-6:
        if np.dot(normal_vector, line_point - plain_point) == 0:
            # The line is coincident with the plane
            return line_point
        else:
            # The line is parallel to the plane and does not intersect it
            return None
    
    # Compute the t parameter
    t = np.dot(normal_vector, plain_point - line_point) / denominator
    
    # Compute the intersection point
    intersection_point = line_point + t * line_direction
    
    return intersection_point

In [417]:
def can_receive_raw(row):
    p1 = points[points['id'] == row['a']]
    p1 = np.array([p1['x'], p1['y'], p1['z']]).reshape(3)
    p2 = points[points['id'] == row['b']]
    p2 = np.array([p2['x'], p2['y'], p2['z']]).reshape(3)
    p3 = points[points['id'] == row['c']]
    p3 = np.array([p3['x'], p3['y'], p3['z']]).reshape(3)
    reflect_normal = np.cross(p2 - p1, p3 - p1)
    reflect_normal /= np.linalg.norm(reflect_normal)
    reflect_direction = incoming - 2 * np.dot(incoming, reflect_normal) * reflect_normal
    reflect_direction /= np.linalg.norm(reflect_direction)
    center = (p1 + p2 + p3) / 3
    inter = intersection(incoming, receiver, center, reflect_direction)
    dist = np.linalg.norm(inter - receiver)
    return dist < 0.5

In [418]:
tris.apply(lambda row: can_receive_raw(row), axis=1).sum() / len(tris)

0.05019305019305019

In [419]:
def can_receive(points_with_new_position, row):
    p1 = points_with_new_position[points_with_new_position['id'] == row['a']]
    p1 = np.array([p1['x'], p1['y'], p1['z']]).reshape(3)
    p2 = points_with_new_position[points_with_new_position['id'] == row['b']]
    p2 = np.array([p2['x'], p2['y'], p2['z']]).reshape(3)
    p3 = points_with_new_position[points_with_new_position['id'] == row['c']]
    p3 = np.array([p3['x'], p3['y'], p3['z']]).reshape(3)
    reflect_normal = np.cross(p2 - p1, p3 - p1)
    reflect_normal /= np.linalg.norm(reflect_normal)
    reflect_direction = incoming - 2 * np.dot(incoming, reflect_normal) * reflect_normal
    reflect_direction /= np.linalg.norm(reflect_direction)
    center = (p1 + p2 + p3) / 3
    inter = intersection(incoming, receiver, center, reflect_direction)
    dist = np.linalg.norm(inter - receiver)
    return dist < 0.5

In [420]:
def count_signal(delta):
    new_ps = points.apply(lambda row: move_to([row['x'], row['y'], row['z']], delta), axis=1).to_list()
    x = [p[0] for p in new_ps]
    y = [p[1] for p in new_ps]
    z = [p[2] for p in new_ps]
    points_with_new_position = pd.DataFrame({
        "id": points["id"],
        "x": x,
        "y": y,
        "z": z,
    })
    return tris.apply(lambda row: can_receive(points_with_new_position, row), axis=1).sum()

In [421]:
ans = 0

In [422]:
# simulated annealing
sim_T = 1000
sim_T_step = 10
iterations = int((sim_T - 1) // sim_T_step)
count = count_signal(ans)
sigma = 0.3
for i in range(iterations):
    next_ans = ans + np.random.normal(0, sigma)
    next_ans = np.clip(next_ans, -0.6, 0.6)
    next_count = count_signal(next_ans)
    if next_count > count or np.random.rand() < np.exp((next_count - count) / sim_T):
        ans = next_ans
        count = next_count
    if i % (iterations // 20) == 0:
        print(f"""
        i: {i}
        ans: {ans}
        count: {count}
        rate: {count / len(points)}
        """)


        i: 0
        ans: -0.29259765730396126
        count: 65
        rate: 0.09393063583815028
        

        i: 4
        ans: -0.23123325503728093
        count: 65
        rate: 0.09393063583815028
        

        i: 8
        ans: 0.06737352165864544
        count: 65
        rate: 0.09393063583815028
        

        i: 12
        ans: -0.5720388975126904
        count: 65
        rate: 0.09393063583815028
        

        i: 16
        ans: 0.4001510032705341
        count: 65
        rate: 0.09393063583815028
        

        i: 20
        ans: 0.6
        count: 65
        rate: 0.09393063583815028
        

        i: 24
        ans: 0.11246067742232985
        count: 65
        rate: 0.09393063583815028
        

        i: 28
        ans: -0.3711044670512
        count: 65
        rate: 0.09393063583815028
        

        i: 32
        ans: -0.24589653982738668
        count: 65
        rate: 0.09393063583815028
        

        i: 36
        ans: -0.54057773